In [0]:
from sklearn.datasets import fetch_california_housing

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
from sklearn.preprocessing import StandardScaler

In [4]:
housing = fetch_california_housing()

In [0]:
X_train_full, X_test, y_train_full, y_test = train_test_split( housing.data, housing.target)

**WHY SPLITTING TRAIN FURTHER**

In [0]:
X_train, X_valid, y_train, y_valid = train_test_split( X_train_full, y_train_full)

In [0]:
scaler = StandardScaler()

In [0]:
X_train = scaler.fit_transform(X_train)

In [0]:
X_valid = scaler.transform(X_valid)

In [0]:
X_test = scaler.transform(X_test)

In [11]:
X_train.shape[1:]

(8,)

In [12]:
X_train.shape

(11610, 8)

In [0]:
import tensorflow as tf
from tensorflow import keras

Using the Sequential API to build, train, evaluate, and use a regression MLP to make predictions is quite similar to what we did for classification. The main differences are the fact that the output layer has a single neuron (since we only want to predict a single value) and uses no activation function, and the loss function is the mean squared error.

In [0]:
model = keras.models.Sequential([ keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]), keras.layers.Dense(1)
])

In [0]:
model.compile(loss="mean_squared_error", optimizer="sgd")

In [0]:
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 0.9013 - val_loss: 3.5360
Epoch 2/20
363/363 [==============================] - 1s 1ms/step - loss: 1.9570 - val_loss: 0.4369
Epoch 3/20
363/363 [==============================] - 1s 1ms/step - loss: 0.4181 - val_loss: 0.4264
Epoch 4/20
363/363 [==============================] - 1s 1ms/step - loss: 0.3929 - val_loss: 0.3864
Epoch 5/20
363/363 [==============================] - 1s 1ms/step - loss: 0.3841 - val_loss: 0.3707
Epoch 6/20
363/363 [==============================] - 1s 1ms/step - loss: 0.3761 - val_loss: 0.3689
Epoch 7/20
363/363 [==============================] - 1s 1ms/step - loss: 0.3746 - val_loss: 0.3671
Epoch 8/20
363/363 [==============================] - 1s 1ms/step - loss: 0.3728 - val_loss: 0.3718
Epoch 9/20
363/363 [==============================] - 1s 1ms/step - loss: 0.3734 - val_loss: 0.4473
Epoch 10/20
363/363 [==============================] - 1s 1ms/step - loss: 0.3760 - val_loss: 0.3607

In [0]:
mse_test = model.evaluate(X_test, y_test)

162/162 [==============================] - 0s 905us/step - loss: 0.3679


In [0]:
X_new = X_test[:3]

In [0]:
y_pred = model.predict(X_new)

**RANDOMIZED SEARCH**

To do this, we need to wrap our Keras models in objects that mimic regular Scikit-Learn regressors

In [0]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
  model = keras.models.Sequential()
  model.add(keras.layers.InputLayer(input_shape=input_shape))
  for layer in range(n_hidden):
    model.add(keras.layers.Dense(n_neurons, activation="relu"))
  model.add(keras.layers.Dense(1))
  optimizer = keras.optimizers.SGD(lr=learning_rate)
  model.compile(loss="mse", optimizer=optimizer)
  return model

This function creates a simple Sequential model for univariate regression (only one output neuron),

In [0]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

**keras.wrappers.scikit_learn.KerasRegressor(build_model)**


---

---





Take "build_model" and make it a keras regressor with functionality of sklearn

*Note that any extra parameter you pass to the fit() method will get passed to the underlying Keras model*

In [16]:
keras_reg.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100
363/363 [==============================] - 1s 3ms/step - loss: 1.3761 - val_loss: 0.7534
Epoch 2/100
363/363 [==============================] - 1s 3ms/step - loss: 0.6575 - val_loss: 0.6205
Epoch 3/100
363/363 [==============================] - 1s 3ms/step - loss: 0.5765 - val_loss: 0.5489
Epoch 4/100
363/363 [==============================] - 1s 3ms/step - loss: 0.5223 - val_loss: 0.5409
Epoch 5/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4937 - val_loss: 0.4956
Epoch 6/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4738 - val_loss: 0.4713
Epoch 7/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4604 - val_loss: 0.4628
Epoch 8/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4488 - val_loss: 0.4559
Epoch 9/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4399 - val_loss: 0.4491
Epoch 10/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4348 - val_lo

In [19]:
mse_test = keras_reg.score(X_test, y_test)
y_pred = keras_reg.predict(X_new)

162/162 [==============================] - 0s 2ms/step - loss: 0.3469


In [0]:
from scipy.stats import reciprocal 
from sklearn.model_selection import RandomizedSearchCV

In [0]:
import numpy as np
param_distribs = {
                 "n_hidden": [0, 1, 2, 3],
                  "n_neurons": np.arange(1, 100),
                  "learning_rate": reciprocal(3e-4, 3e-2),
                  }

In [0]:
?reciprocal()

In [0]:
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)

In [31]:
rnd_search_cv.fit(
                  X_train, y_train, epochs=100,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)]
                  )

Streaming output truncated to the last 5000 lines.
Epoch 7/100
242/242 [==============================] - 1s 3ms/step - loss: 0.4067 - val_loss: 0.4154
Epoch 8/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3974 - val_loss: 0.4083
Epoch 9/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3883 - val_loss: 0.4006
Epoch 10/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3822 - val_loss: 0.3960
Epoch 11/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3777 - val_loss: 0.3885
Epoch 12/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3729 - val_loss: 0.3858
Epoch 13/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3698 - val_loss: 0.3810
Epoch 14/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3673 - val_loss: 0.3764
Epoch 15/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3643 - val_loss: 0.3835
Epoch 16/100
242/242 [============

RuntimeError: ignored

This is identical to what we did in Chapter 2, except here we pass extra parameters to the fit() method, and they get relayed to the underlying Keras models.

Note that RandomizedSearchCV uses K-fold cross-validation, so it does not use X_valid and y_valid, which are only used for early stopping.

**NOTE**


---

This is identical to what we did in Chapter 2, except here we pass extra parameters to the fit() method, and they get relayed to the underlying Keras models. Note that RandomizedSearchCV uses K-fold cross-validation, so it does not use X_valid and y_valid, which are only used for early stopping.

In [32]:
rnd_search_cv.best_params_

{'learning_rate': 0.013904032607790832, 'n_hidden': 3, 'n_neurons': 47}

In [33]:
rnd_search_cv.best_score_

-0.3139094611008962

In [34]:
model = rnd_search_cv.best_estimator_.model

AttributeError: ignored